In [1]:
import os
import sys
import tyro
import time
import random
import warnings

import numpy as np
import torch

oc_atari_dir = os.getenv("OC_ATARI_DIR")

if oc_atari_dir is not None:
    a = os.path.join(Path(__file__), oc_atari_dir)
    sys.path.insert(1, a)

sys.path.append("..")
has_agent= False

In [2]:
env_id = "ALE/Pong-v5"
obs_mode = "obj"
pth = "/data/oc_cleanrl/cleanrl/wandb/run-20241010_060604-pzk9e36i/files/DQNLv2.cleanrl_model"
architecture = "DQNLv2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    

In [14]:
from ocatari.core import OCAtari
env = OCAtari(
    env_id, hud=False, render_mode="rgb_array",
        render_oc_overlay=False, obs_mode=obs_mode, 
        # logger=logger, feature_func=feature_func,
        # buffer_window_size=window_size
)

obs, info = env.reset()

Using OBJ State Representation


In [15]:
if architecture == "OCT":
    from architectures.transformer import OCTransformer as Agent
    agent = Agent(env, emb_dim, num_heads, num_blocks, device).to(device)
elif architecture == "VIT":
    from architectures.transformer import VIT as Agent
    agent = Agent(env, emb_dim, num_heads, num_blocks,
                    patch_size, buffer_window_size, device).to(device)
elif architecture == "VIT2":
    from architectures.transformer import SimpleViT2 as Agent
    agent = Agent(env, emb_dim, num_heads, num_blocks,
                    patch_size, buffer_window_size, device).to(device)
elif architecture == "MobileVit":
    from architectures.transformer import MobileVIT as Agent
    agent = Agent(env, emb_dim, num_heads, num_blocks,
                    patch_size, buffer_window_size, device).to(device)
elif architecture == "MobileVit2":
    from architectures.transformer import MobileViT2 as Agent
    agent = Agent(env, emb_dim, num_heads, num_blocks,
                    patch_size, buffer_window_size, device).to(device)
elif architecture == "PPO":
    from architectures.ppo import PPODefault as Agent
    agent = Agent(env, device).to(device)
else:
    from architectures.ppo import PPO_Obj as Agent
    agent = Agent(env, device).to(device)


In [16]:
ckpt = torch.load(pth, map_location=torch.device('cpu'))
agent.load_state_dict(ckpt["model_weights"]) 
has_agent = True 

In [17]:
import numpy as np
import matplotlib.pyplot as plt
def print_obs(obs):
    
    # Display the array as an image
    plt.imshow(obs, cmap='gray')  # Use 'gray' colormap for 2D arrays
    plt.axis('off')
    plt.show()  

In [18]:
episode_rewards = []

for i in range(21):
    done = False
    crew = 0
    j = 0
    while not done:
        if has_agent:
            obs = torch.from_numpy(obs).to(device)
            obs = obs.unsqueeze(0)
            action, _, _, _ = agent.get_action_and_value(obs)
        else:
            action = env.action_space.sample() # random moves
        
        obs, reward, terminated, truncated, info = env.step(action)
        crew += reward
        j += 1
        
        if terminated or truncated:
            print(f"{env_id}: Reward: {crew}, Framses:", info["episode_frame_number"], f"w/o Frameskip: {j}, Episode: {i}")
            #run.log({f"{opts.game}_reward": crew, f"{opts.game}_episode_length": info["episode_frame_number"]})
            observation, info = env.reset()
            done = True
            episode_rewards.append(crew)

print()
mean_reward = np.mean(episode_rewards)
std_reward = np.std(episode_rewards)

print(f"{mean_reward} +- {std_reward}")
env.close()

{'lives': 0, 'episode_frame_number': 18270, 'frame_number': 18270}
4568
ALE/Pong-v5: Reward: 7.0, Length: 18270 Episode: 0
{'lives': 0, 'episode_frame_number': 12401, 'frame_number': 30671}
3101
ALE/Pong-v5: Reward: 17.0, Length: 12401 Episode: 1
{'lives': 0, 'episode_frame_number': 13878, 'frame_number': 44549}
3470
ALE/Pong-v5: Reward: 9.0, Length: 13878 Episode: 2


KeyboardInterrupt: 